In [4]:

import time
import pandas as pd
import requests
import re
import numpy as np
from bs4 import BeautifulSoup
# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
#selenium packages
#selenium packages
#pip install webdriver-manager
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [247]:
def getReviews(soup,i):
    print("Doing for page",i)
    articleDiv=soup.find_all("article",{"class" :"Review_review__1efy6"})
    #print(len(articleDiv))
    roleName=''
    companyName=''
    companySize=''
    score=''
    for article in articleDiv:
        infoDiv=article.find("div",{"class" :"Byline_byline__3B_xU"})
        divChild=infoDiv.findChildren("div" , recursive=False)
        aChild=infoDiv.findChildren("a" , recursive=False)
        for child in divChild:
            tempList=child.findChildren("div" , recursive=True)
            #print(tempList[1].text)
            compDetails=child.findChildren("span" , recursive=True)
            #print(compDetails[0].text)
            if(len(aChild))==0:
                roleName=tempList[0].text
            else:
                 roleName=tempList[1].text
            detailsList=compDetails[0].text.split(",")
            companyName=detailsList[0]
            companySize=detailsList[1]
            tempScore=article.find('span',{'class':'ReviewScore_score-value__1ZgYG'}).text.split("out")
            score=tempScore[0].strip(" ")
            #print(".........")
        reviewEle =article.find_all("div",{"class" :"question-bullets"})
        for review in reviewEle:
            psReviewEle =review.find_all("ul",{"data-slug" :"things-done-well"})
            #print("length of posreviewEle",len(psReviewEle))
            for ulEle in psReviewEle:
                #print(ulEle)
                divEle =ulEle.find_all("div")
                for div in divEle:
                    tempDict={}
                    #print(div)
                    reviewText=div.get_text()
                    tempDict['review']=reviewText
                    tempDict['class']='Positive'
                    tempDict['companyName']=companyName
                    tempDict['companySize']=companySize
                    tempDict['roleName']=roleName
                    tempDict['score']=score
                    reviewsList.append(tempDict)

            negReviewEle =review.find_all("ul",{"data-slug" :"things-done-poorly"})
            #print("length of negreviewEle",len(negReviewEle))
            for ulEle in negReviewEle:
                #print(ulEle)
                divEle =ulEle.find_all("div")
                for div in divEle:
                    tempDict={}
                    #print(div)
                    reviewText=div.get_text()
                    tempDict['review']=reviewText
                    tempDict['class']='Negative'
                    tempDict['companyName']=companyName
                    tempDict['companySize']=companySize
                    tempDict['roleName']=roleName
                    tempDict['score']=score
                    reviewsList.append(tempDict)


In [248]:
baseUrl="https://www.trustradius.com/products/appdynamics/reviews"
from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')
driver = webdriver.Chrome(ChromeDriverManager(version="95.0.4638.69").install())
reviewsList=[]
for i in range(1,3):
    if(i==1):
        driver.get(baseUrl)
        
    else:
        driver.execute_script("window.open('about:blank','secondtab');")
        driver.switch_to.window("secondtab")
        time.sleep(3)
        # In the second tab, it opens geeksforgeek
        driver.get('https://www.trustradius.com/products/appdynamics/reviews?qs=pros-and-cons&f=25')
        time.sleep(10)
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')
    getReviews(soup,i)
        
driver.close()



====== WebDriver manager ======
Driver [C:\Users\jhash\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


Doing for page 1
Doing for page 2


In [249]:
print("Total reviews are",len(reviewsList))

Total reviews are 160


In [252]:
df=pd.DataFrame(reviewsList,columns=['review','class','companyName','companySize','roleName','score'])
df.to_csv("reviewsV2.csv")
df.head()

,review,class,companyName,companySize,roleName,score
0,AppDynamics community support is really strong...,Positive,Synechron (Information Technology & Services,5001-10,Senior Associate Technology,8
1,Documentation maintained with each version sup...,Positive,Synechron (Information Technology & Services,5001-10,Senior Associate Technology,8
2,Trial product is really good as they provide t...,Positive,Synechron (Information Technology & Services,5001-10,Senior Associate Technology,8
3,In terms of dashboard I believe there is plent...,Negative,Synechron (Information Technology & Services,5001-10,Senior Associate Technology,8
4,Synthetic monitoring part is still complex and...,Negative,Synechron (Information Technology & Services,5001-10,Senior Associate Technology,8
